In [1]:
import statsmodels
import scipy as sc
import numpy as np
import pandas as pd
import statsmodels.formula.api as smf
import statsmodels.stats.api as sms
from statsmodels.graphics.regressionplots import plot_leverage_resid2
import matplotlib.pyplot as plt
from collections import Counter

/anaconda2/lib/python2.7/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [2]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


Давайте проанализируем данные опроса 4361 женщин из Ботсваны:

botswana.tsv
О каждой из них мы знаем:

сколько детей она родила (признак ceb)
возраст (age)
длительность получения образования (educ)
религиозная принадлежность (religion)
идеальное, по её мнению, количество детей в семье (idlnchld)
была ли она когда-нибудь замужем (evermarr)
возраст первого замужества (agefm)
длительность получения образования мужем (heduc)
знает ли она о методах контрацепции (knowmeth)
использует ли она методы контрацепции (usemeth)
живёт ли она в городе (urban)
есть ли у неё электричество, радио, телевизор и велосипед (electric, radio, tv, bicycle)
Давайте научимся оценивать количество детей ceb по остальным признакам.

Загрузите данные и внимательно изучите их. Сколько разных значений принимает признак religion?

In [3]:
raw = pd.read_csv("botswana.tsv", sep="\t", index_col=False) 
raw.head()

,ceb,age,educ,religion,idlnchld,knowmeth,usemeth,evermarr,agefm,heduc,urban,electric,radio,tv,bicycle
0,0,18,10,catholic,4.0,1.0,1.0,0,NaN,NaN,1,1.0,1.0,1.0,1.0
1,2,43,11,protestant,2.0,1.0,1.0,1,20.0,14.0,1,1.0,1.0,1.0,1.0
2,0,49,4,spirit,4.0,1.0,0.0,1,22.0,1.0,1,1.0,1.0,0.0,0.0
3,0,24,12,other,2.0,1.0,0.0,0,NaN,NaN,1,1.0,1.0,1.0,1.0
4,3,32,13,other,3.0,1.0,1.0,1,24.0,12.0,1,1.0,1.0,1.0,1.0


In [4]:
Counter(raw.religion)

Counter({'catholic': 447, 'other': 1080, 'protestant': 993, 'spirit': 1841})

Во многих признаках есть пропущенные значения. Сколько объектов из 4361 останется, если выбросить все, содержащие пропуски?

In [5]:
shape(raw.dropna())

(1834, 15)

Давайте используем этот метод для обработки пропусков в agefm и heduc.

- Создайте признак nevermarr, равный единице там, где в agefm пропуски.
- Удалите признак evermarr — в сумме с nevermarr он даёт константу, значит, в нашей матрице XX будет мультиколлинеарность.
- Замените NaN в признаке agefm на C agefm = 0.
- У объектов, где nevermarr = 1, замените NaN в признаке heduc на c heduc  =−1 (ноль использовать нельзя, так как он уже встречается у некоторых объектов выборки).
Сколько осталось пропущенных значений в признаке heduc?

In [6]:
#another way to do the same
#raw.loc[raw['agefm'].isnull(),'nevermarr'] = 1
#raw.loc[raw['agefm'].notnull(),'nevermarr'] = 0

raw['nevermarr'] = raw['agefm'].apply(lambda x : 1 if pd.isnull(x) else 0)
raw = raw.drop(['evermarr'], axis = 1)
raw['agefm'].fillna(0,inplace=True)

for index, row in raw.iterrows():
    if (row['nevermarr'] == 1) and (pd.isnull(row['heduc'])):
      raw.loc[index,'heduc'] = -1
    
len(raw.loc[raw['heduc'].isnull()].axes[0])

123

Избавимся от оставшихся пропусков.

Для признаков idlnchld, heduc и usemeth проведите операцию, аналогичную предыдущей: создайте индикаторы пропусков по этим признакам (idlnchld_noans, heduc_noans, usemeth_noans), замените пропуски на нехарактерные значения c idlnchld =−1, c heduc =−2 (значение -1 мы уже использовали), c usemeth =−1).

Остались только пропуски в признаках knowmeth, electric, radio, tv и bicycle. Их очень мало, так что удалите объекты, на которых их значения пропущены.

Какого размера теперь наша матрица данных? Умножьте количество строк на количество всех столбцов (включая отклик ceb).

In [7]:
raw['idlnchld_noans'] = raw['idlnchld'].apply(lambda x : 1 if pd.isnull(x) else 0)
raw['heduc_noans'] = raw['heduc'].apply(lambda x : 1 if pd.isnull(x) else 0)
raw['usemeth_noans'] = raw['usemeth'].apply(lambda x : 1 if pd.isnull(x) else 0)

raw['idlnchld']=raw['idlnchld'].fillna(-1)
raw['heduc']=raw['heduc'].fillna(-2)
raw['usemeth']=raw['usemeth'].fillna(-1)

data = raw.dropna()
data.shape[0]*data.shape[1]

78264

Постройте регрессию количества детей ceb на все имеющиеся признаки методом smf.ols, как в разобранном до этого примере. Какой получился коэффициент детерминации R^2? Округлите до трёх знаков после десятичной точки.


Обратите внимание, что для признака religion в модели автоматически создалось несколько бинарных фиктивных переменных. Сколько их?

In [8]:
m1 = smf.ols('ceb ~ age + educ + religion + idlnchld + knowmeth + usemeth + agefm + heduc + urban + '\
             'electric + radio + tv + bicycle + nevermarr + idlnchld_noans + heduc_noans + usemeth_noans',
             data = data)
fitted = m1.fit()
print fitted.summary()

                            OLS Regression Results                            
Dep. Variable:                    ceb   R-squared:                       0.644
Model:                            OLS   Adj. R-squared:                  0.643
Method:                 Least Squares   F-statistic:                     412.5
Date:                Sun, 29 Jul 2018   Prob (F-statistic):               0.00
Time:                        19:52:24   Log-Likelihood:                -7732.1
No. Observations:                4348   AIC:                         1.550e+04
Df Residuals:                    4328   BIC:                         1.563e+04
Df Model:                          19                                         
Covariance Type:            nonrobust                                         
                             coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------
Intercept                 -1

Проверьте критерием Бройша-Пагана гомоскедастичность ошибки в построенной модели. Выполняется ли она?

In [9]:
p = sms.het_breuschpagan(fitted.resid, fitted.model.exog)[1]
p

1.1452927633437192e-225

Удалите из модели незначимые признаки religion, radio и tv. Проверьте гомоскедастичность ошибки, при необходимости сделайте поправку Уайта.

Не произошло ли значимого ухудшения модели после удаления этой группы признаков? Проверьте с помощью критерия Фишера. Чему равен его достигаемый уровень значимости? Округлите до четырёх цифр после десятичной точки.

Если достигаемый уровень значимости получился маленький, верните все удалённые признаки; если он достаточно велик, оставьте модель без религии, тв и радио.

In [10]:
m2 = smf.ols('ceb ~ age + educ + idlnchld + knowmeth + usemeth + agefm + heduc + urban + '\
             'electric + bicycle + nevermarr + idlnchld_noans + heduc_noans + usemeth_noans',
             data = data)
fitted = m2.fit(cov_type='HC1')
print fitted.summary()

                            OLS Regression Results                            
Dep. Variable:                    ceb   R-squared:                       0.644
Model:                            OLS   Adj. R-squared:                  0.643
Method:                 Least Squares   F-statistic:                     463.4
Date:                Sun, 29 Jul 2018   Prob (F-statistic):               0.00
Time:                        19:53:05   Log-Likelihood:                -7734.5
No. Observations:                4348   AIC:                         1.550e+04
Df Residuals:                    4333   BIC:                         1.559e+04
Df Model:                          14                                         
Covariance Type:                  HC1                                         
                     coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept         -1.0698      0.258     -4.

In [11]:
print "F=%f, p=%f, k1=%f" % m1.fit().compare_f_test(m2.fit())

F=0.919236, p=0.467231, k1=5.000000


Признак usemeth_noans значим по критерию Стьюдента, то есть, при его удалении модель значимо ухудшится. Но вообще-то отдельно его удалять нельзя: из-за того, что мы перекодировали пропуски в usemeth произвольно выбранным значением c usemeth =−1, удалять usemeth_noans и usemeth можно только вместе.

Удалите из текущей модели usemeth_noans и usemeth. Проверьте критерием Фишера гипотезу о том, что качество модели не ухудшилось. Введите номер первой значащей цифры в достигаемом уровне значимости (например, если вы получили 5.5\times10^{-8}5.5×10 
−8, нужно ввести 8).

Если достигаемый уровень значимости получился маленький, верните удалённые признаки; если он достаточно велик, оставьте модель без usemeth и usemeth_noans.

In [12]:
m3 = smf.ols('ceb ~ age + educ + idlnchld + knowmeth + agefm + heduc + urban + '\
             'electric + bicycle + nevermarr + idlnchld_noans + heduc_noans',
             data = data)
fitted = m3.fit(cov_type='HC1')
print fitted.summary()

                            OLS Regression Results                            
Dep. Variable:                    ceb   R-squared:                       0.629
Model:                            OLS   Adj. R-squared:                  0.628
Method:                 Least Squares   F-statistic:                     396.4
Date:                Sun, 29 Jul 2018   Prob (F-statistic):               0.00
Time:                        19:53:12   Log-Likelihood:                -7825.7
No. Observations:                4348   AIC:                         1.568e+04
Df Residuals:                    4335   BIC:                         1.576e+04
Df Model:                          12                                         
Covariance Type:                  HC1                                         
                     coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept         -1.1931      0.262     -4.

In [13]:
F, p, k1 = m2.fit().compare_f_test(m3.fit())
p

3.1552009480371243e-40

In [14]:
0.5266 + 0.0705

0.5971